## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,-3.98,77,0,20.71,haze
1,1,Ushuaia,AR,-54.8000,-68.3000,55.02,81,100,9.22,overcast clouds
2,2,Maloy,NO,61.9353,5.1136,42.73,75,100,8.16,overcast clouds
3,3,Saskylakh,RU,71.9167,114.0833,-21.50,100,100,3.02,overcast clouds
4,4,Umm Kaddadah,SD,13.6017,26.6876,78.12,10,19,18.37,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Umm Kaddadah,SD,13.6017,26.6876,78.12,10,19,18.37,few clouds
6,6,Nortelandia,BR,-14.4547,-56.8028,79.23,80,97,5.57,overcast clouds
17,17,Qui Nhon,VN,13.7667,109.2333,79.12,91,20,3.58,few clouds
18,18,Rikitea,PF,-23.1203,-134.9692,78.76,75,4,13.38,clear sky
20,20,Port-Gentil,GA,-0.7193,8.7815,82.31,81,100,12.39,moderate rain
21,21,Geraldton,AU,-28.7667,114.6000,73.81,88,40,4.61,scattered clouds
22,22,Bambous Virieux,MU,-20.3428,57.7575,81.21,83,40,4.61,scattered clouds
26,26,Cape Town,ZA,-33.9258,18.4232,76.12,60,0,14.97,clear sky
28,28,Moerai,PF,-22.4333,-151.3333,79.84,78,91,9.31,light rain
33,33,Enterprise,US,36.0252,-115.2419,76.86,20,40,3.44,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Umm Kaddadah,SD,78.12,few clouds,13.6017,26.6876,
6,Nortelandia,BR,79.23,overcast clouds,-14.4547,-56.8028,
17,Qui Nhon,VN,79.12,few clouds,13.7667,109.2333,
18,Rikitea,PF,78.76,clear sky,-23.1203,-134.9692,
20,Port-Gentil,GA,82.31,moderate rain,-0.7193,8.7815,
21,Geraldton,AU,73.81,scattered clouds,-28.7667,114.6000,
22,Bambous Virieux,MU,81.21,scattered clouds,-20.3428,57.7575,
26,Cape Town,ZA,76.12,clear sky,-33.9258,18.4232,
28,Moerai,PF,79.84,light rain,-22.4333,-151.3333,
33,Enterprise,US,76.86,scattered clouds,36.0252,-115.2419,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Nortelandia,BR,79.23,overcast clouds,-14.4547,-56.8028,Hotel Karysmã
17,Qui Nhon,VN,79.12,few clouds,13.7667,109.2333,Khách sạn Hải Âu Quy Nhơn - Seagull Hotel
18,Rikitea,PF,78.76,clear sky,-23.1203,-134.9692,People ThankYou
20,Port-Gentil,GA,82.31,moderate rain,-0.7193,8.7815,Fengshui Residence
21,Geraldton,AU,73.81,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))